In [2]:
# from langchain_community.vectorstores import Qdrant
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# # from langchain.chains import RetrievalQA
# # from langchain.schema import Document
# from langchain_core.documents import Document
# # from qdrant_client import QdrantClient
# from qdrant_client import QdrantClient, models as qm
# from qdrant_client.http import models
import json
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.retrievers.multi_vector import MultiVectorRetriever
# from langchain.retrievers import EnsembleRetriever
# from langchain_community.retrievers import BM25Retriever, EnsembleRetriever
from qdrant_client import QdrantClient
# from langchain_community.vectorstores import BM25Retriever
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
from qdrant_client.models import VectorParams, Distance, SparseVectorParams

In [3]:
QDRANT_URL = "http://127.0.0.1:6333"
COLLECTION_NAME = "hybrid_docs"
DATA_PATH = "../data/processed/proc_docx.jsonl"
EMBED_MODEL_NAME = 'intfloat/multilingual-e5-large'
EMBED_DIM = 1024

In [4]:
docs = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data.get("text", "")
        meta = {
            "doc_id": data.get("doc_id", ""),
            "file_path": data.get("file_path", "")
        }
        if text.strip():
            docs.append({"text": text, "metadata": meta})

In [5]:
len(docs)

31

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts, metadatas = [], []
for doc in docs:
    chunks = splitter.split_text(doc["text"])
    texts.extend(chunks)
    metadatas.extend([doc["metadata"]] * len(chunks))

In [7]:
len(texts), len(metadatas)

(53, 53)

In [8]:
texts = ['passage: ' + t for t in texts]

In [9]:
# emb = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
embedder = SentenceTransformer(EMBED_MODEL_NAME)

In [10]:
client = QdrantClient(url=QDRANT_URL)

In [11]:
client.recreate_collection(
    collection_name="hybrid_docs",
    vectors_config={
        "dense": VectorParams(size=1024, distance=Distance.COSINE),
        "sparse": SparseVectorParams(),
    },
)

/tmp/ipykernel_14634/3340005599.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


ValidationError: 5 validation errors for CreateCollection
vectors.VectorParams.size
  Field required [type=missing, input_value={'dense': VectorParams(si...ex=None, modifier=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
vectors.VectorParams.distance
  Field required [type=missing, input_value={'dense': VectorParams(si...ex=None, modifier=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
vectors.VectorParams.dense
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorParams(size=1024, d...multivector_config=None), input_type=VectorParams]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden
vectors.VectorParams.sparse
  Extra inputs are not permitted [type=extra_forbidden, input_value=SparseVectorParams(index=None, modifier=None), input_type=SparseVectorParams]
    For further information visit https://errors.pydantic.dev/2.12/v/

In [ ]:
points = []
for i, text in enumerate(texts):
    dense_vec = embedder.encode(text).tolist()
    # sparse можно просто указать пустым (Qdrant сам посчитает BM25 на лету)
    points.append(
        PointStruct(
            id=i,
            vector={"dense": dense_vec},  # без sparse
            payload={"text": text}
        )
    )

client.upsert(collection_name=COLLECTION_NAME, points=points)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `hybrid_docs` doesn\'t exist!"},"time":0.000049021}'

In [ ]:
from qdrant_client.models import NamedVector, SparseVector

query = "как работает гибридный поиск в Qdrant?"

dense_vec = embedder.encode(query).tolist()

results = client.query_points(
    collection_name="hybrid_docs",
    prefetch=[
        {"dense": {"vector": dense_vec}},
        {"sparse": {"text": query}},  # <- Qdrant сам токенизирует и применяет BM25
    ],
    limit=5,
)


In [ ]:
results = client.query_points(
    collection_name="hybrid_docs",
    prefetch=[
        {"dense": {"vector": dense_vec}},
        {"sparse": {"text": query}},
    ],
    limit=5,
    query_filter=None,
    search_params={"alpha": 0.5},  # баланс dense/sparse
)
